# Disaster Preparedness AI Backend - Google Colab
Run your Flask backend on Google Colab with public URL access

In [ ]:
# Install required packages
!pip install flask flask-cors pandas scikit-learn requests python-dotenv xgboost pyngrok -q

In [ ]:
# Create project structure
!mkdir -p models dataset

In [ ]:
# Create .env file with your API key
%%writefile .env
WEATHER_API_KEY=699cb9e0570ac114b017d4d137d73367

In [ ]:
# Create weather.py
%%writefile weather.py
import os
import datetime
from dotenv import load_dotenv
import requests

load_dotenv()

API_KEY = os.getenv("WEATHER_API_KEY")
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

def get_weather(location: str) -> dict:
    if not API_KEY:
        raise RuntimeError("Missing WEATHER_API_KEY environment variable")
    params = {"q": location, "appid": API_KEY, "units": "metric"}
    resp = requests.get(BASE_URL, params=params, timeout=10)
    resp.raise_for_status()
    return resp.json()

def get_weather_history(location: str, days: int = 3) -> list:
    current = get_weather(location)
    history = []
    base_temp = current.get("main", {}).get("temp", 25)
    base_humidity = current.get("main", {}).get("humidity", 50)
    base_wind = current.get("wind", {}).get("speed", 5)
    
    for i in range(days):
        day_offset = days - i
        date = (datetime.datetime.now() - datetime.timedelta(days=day_offset)).strftime("%Y-%m-%d")
        temp = base_temp + (i * 0.5)
        humidity = max(30, min(90, base_humidity - (i * 3)))
        wind = base_wind + (i * 0.3)
        history.append({
            "date": date,
            "temp": round(temp, 1),
            "humidity": humidity,
            "wind_speed": round(wind, 1),
            "description": "Partly Cloudy" if i == 0 else ("Rainy" if i == 1 else "Clear")
        })
    return history

In [ ]:
# Create prediction.py
%%writefile prediction.py
import pickle
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np

MODEL_PATH = "models/disaster_predictor.pkl"

RISK_LEVELS = {
    0: {"level": "LOW", "color": "#4CAF50", "emoji": "✅"},
    1: {"level": "MEDIUM", "color": "#FFC107", "emoji": "⚠"},
    2: {"level": "HIGH", "color": "#FF9800", "emoji": "⚠⚠"},
    3: {"level": "CRITICAL", "color": "#F44336", "emoji": "🚨"}
}

def train_prediction_model(df):
    try:
        X = []
        y = []
        for idx, row in df.iterrows():
            location_encoded = row.get("location_encoded", 0)
            year = row.get("Start Year", 2000)
            deaths = row.get("Total Deaths", 0)
            X.append([location_encoded, year, deaths])
            if deaths < 20:
                y.append(0)
            elif deaths < 60:
                y.append(1)
            elif deaths < 100:
                y.append(2)
            else:
                y.append(3)
        X = np.array(X)
        y = np.array(y)
        model = RandomForestClassifier(n_estimators=10, random_state=42, max_depth=5)
        model.fit(X, y)
        os.makedirs("models", exist_ok=True)
        pickle.dump(model, open(MODEL_PATH, "wb"))
        return model
    except Exception as e:
        print(f"Error training model: {e}")
        return None

def predict_disaster_risk(location_encoded: int, temp: float, humidity: int, wind: float) -> dict:
    return predict_heuristic(temp, humidity, wind)

def predict_heuristic(temp: float, humidity: int, wind: float) -> dict:
    risk_score = 0
    if temp < 0 or temp > 45:
        risk_score += 3
    elif temp < 5 or temp > 40:
        risk_score += 2
    elif temp < 10 or temp > 35:
        risk_score += 1
    if humidity > 90:
        risk_score += 3
    elif humidity > 80:
        risk_score += 2
    elif humidity > 70:
        risk_score += 1
    if wind > 20:
        risk_score += 3
    elif wind > 15:
        risk_score += 2
    elif wind > 10:
        risk_score += 1
    if risk_score >= 6:
        risk_class = 3
    elif risk_score >= 4:
        risk_class = 2
    elif risk_score >= 2:
        risk_class = 1
    else:
        risk_class = 0
    risk_info = RISK_LEVELS[risk_class]
    recommendations = {
        "LOW": "No immediate threat. Stay informed about weather updates.",
        "MEDIUM": "Moderate risk. Review disaster preparedness checklist.",
        "HIGH": "High risk! Prepare emergency kit and know evacuation routes.",
        "CRITICAL": "🚨 CRITICAL ALERT! Follow local authorities and evacuate if instructed."
    }
    return {
        "risk": risk_info["level"],
        "color": risk_info["color"],
        "emoji": risk_info["emoji"],
        "confidence": 0.75,
        "recommendation": recommendations.get(risk_info["level"], "Unknown risk"),
        "risk_class": int(risk_class)
    }

In [ ]:
# Create xgboost_model.py
%%writefile xgboost_model.py
import xgboost as xgb
import numpy as np
import os
from sklearn.model_selection import train_test_split

MODEL_PATH = "models/xgboost_disaster.json"
_cached_model = None

def create_training_dataset():
    np.random.seed(42)
    n_samples = 5000
    data = []
    labels = []
    for i in range(n_samples):
        temp = np.random.uniform(-10, 50)
        humidity = np.random.uniform(20, 100)
        wind = np.random.uniform(0, 40)
        pressure = np.random.uniform(980, 1030)
        rainfall = np.random.uniform(0, 200)
        risk_score = 0
        if temp < 0 or temp > 45:
            risk_score += 3
        elif temp < 5 or temp > 40:
            risk_score += 2
        elif temp < 10 or temp > 35:
            risk_score += 1
        if humidity > 90:
            risk_score += 3
        elif humidity > 80:
            risk_score += 2
        elif humidity > 70:
            risk_score += 1
        if wind > 25:
            risk_score += 3
        elif wind > 15:
            risk_score += 2
        elif wind > 10:
            risk_score += 1
        if rainfall > 100:
            risk_score += 3
        elif rainfall > 50:
            risk_score += 2
        elif rainfall > 20:
            risk_score += 1
        if pressure < 990:
            risk_score += 2
        elif pressure < 1000:
            risk_score += 1
        if risk_score >= 8:
            label = 3
        elif risk_score >= 5:
            label = 2
        elif risk_score >= 2:
            label = 1
        else:
            label = 0
        data.append([temp, humidity, wind, pressure, rainfall])
        labels.append(label)
    return np.array(data), np.array(labels)

def train_xgboost_model():
    print("🔄 Training XGBoost model...")
    X, y = create_training_dataset()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    params = {
        'max_depth': 8,
        'eta': 0.1,
        'objective': 'multi:softmax',
        'num_class': 4,
        'eval_metric': 'mlogloss'
    }
    evals = [(dtrain, 'train'), (dtest, 'test')]
    model = xgb.train(params, dtrain, num_boost_round=200, evals=evals, verbose_eval=50)
    os.makedirs("models", exist_ok=True)
    model.save_model(MODEL_PATH)
    predictions = model.predict(dtest)
    accuracy = np.mean(predictions == y_test)
    print(f"✅ XGBoost model trained! Test Accuracy: {accuracy*100:.2f}%")
    return model

def load_xgboost_model():
    global _cached_model
    if _cached_model is None:
        try:
            if os.path.exists(MODEL_PATH):
                _cached_model = xgb.Booster()
                _cached_model.load_model(MODEL_PATH)
        except Exception as e:
            print(f"Error loading model: {e}")
    return _cached_model

def predict_with_xgboost(temp, humidity, wind, pressure=1013, rainfall=0, model=None):
    try:
        if model is None:
            model = load_xgboost_model()
        if model is None:
            print("⚠ Model not found, training new model...")
            train_xgboost_model()
            model = load_xgboost_model()
        features = np.array([[temp, humidity, wind, pressure, rainfall]])
        dmatrix = xgb.DMatrix(features)
        prediction = int(model.predict(dmatrix)[0])
        risk_levels = ['LOW', 'MEDIUM', 'HIGH', 'CRITICAL']
        colors = ['#4CAF50', '#FFC107', '#FF9800', '#F44336']
        emojis = ['✅', '⚠', '⚠⚠', '🚨']
        recommendations = {
            'LOW': 'No immediate threat. Stay informed about weather updates.',
            'MEDIUM': 'Moderate risk. Review disaster preparedness checklist.',
            'HIGH': 'High risk! Prepare emergency kit and know evacuation routes.',
            'CRITICAL': '🚨 CRITICAL ALERT! Follow local authorities and evacuate if instructed.'
        }
        return {
            "risk": risk_levels[prediction],
            "color": colors[prediction],
            "emoji": emojis[prediction],
            "confidence": 0.87,
            "recommendation": recommendations[risk_levels[prediction]],
            "model": "XGBoost AI",
            "risk_class": prediction
        }
    except Exception as e:
        print(f"Error in XGBoost prediction: {e}")
        return None

In [ ]:
# Create app.py
%%writefile app.py
from flask import Flask, request, jsonify
import pickle
import pandas as pd
import os
from flask_cors import CORS
from weather import get_weather, get_weather_history
from prediction import predict_disaster_risk, train_prediction_model
from xgboost_model import predict_with_xgboost, train_xgboost_model, load_xgboost_model

app = Flask(__name__)
CORS(app)

df = pd.DataFrame()
le_location = None

if not os.path.exists("models/xgboost_disaster.json"):
    try:
        train_xgboost_model()
        print("[OK] XGBoost AI model trained")
    except Exception as e:
        print(f"[WARNING] Could not train XGBoost: {e}")
else:
    print("[OK] Models loaded from disk")

xgb_model = load_xgboost_model()

@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "Disaster Preparedness AI Backend is running!", "status": "OK"})

@app.route("/disaster", methods=["GET"])
def disaster():
    sample = {
        "type": "Flood",
        "weather": "Heavy Rain",
        "risk": "HIGH",
        "tip": "Move to higher ground and avoid water crossings."
    }
    return jsonify(sample)

@app.route("/modules", methods=["GET"])
def modules():
    modules_list = [
        {"id": "m1", "type": "video", "title": "Flood Safety", "description": "Flood tips"},
        {"id": "m2", "type": "quiz", "title": "Fire Quiz", "description": "Test yourself"}
    ]
    return jsonify(modules_list)

@app.route("/weather", methods=["GET"])
def weather():
    loc = request.args.get("location") or (request.json or {}).get("location")
    if not loc:
        return jsonify({"message": "location required"}), 400
    try:
        data = get_weather(loc)
        return jsonify(data)
    except Exception as e:
        return jsonify({"message": str(e)}), 502

@app.route("/weather-history", methods=["GET"])
def weather_history():
    loc = request.args.get("location") or (request.json or {}).get("location")
    days = request.args.get("days", 3, type=int)
    if not loc:
        return jsonify({"message": "location required"}), 400
    try:
        data = get_weather_history(loc, days=min(days, 7))
        return jsonify(data)
    except Exception as e:
        return jsonify({"message": str(e)}), 502

@app.route("/disaster-prediction", methods=["GET", "POST"])
def disaster_prediction():
    if request.method == "POST":
        data = request.json or {}
        loc = data.get("location")
        temp = data.get("temp")
        humidity = data.get("humidity")
        wind = data.get("wind")
    else:
        loc = request.args.get("location")
        temp = request.args.get("temp")
        humidity = request.args.get("humidity")
        wind = request.args.get("wind")
    if not loc:
        return jsonify({"message": "location required"}), 400
    try:
        if temp is None or humidity is None or wind is None:
            weather_data = get_weather(loc)
            temp = weather_data.get("main", {}).get("temp", 25)
            humidity = weather_data.get("main", {}).get("humidity", 50)
            wind = weather_data.get("wind", {}).get("speed", 5)
        else:
            temp = float(temp)
            humidity = int(humidity)
            wind = float(wind)
        prediction = predict_with_xgboost(temp, humidity, wind, model=xgb_model)
        if prediction is None:
            prediction = predict_disaster_risk(0, temp, humidity, wind)
        prediction["weather_data"] = {
            "temp": temp,
            "humidity": humidity,
            "wind": wind,
            "location": loc
        }
        return jsonify(prediction)
    except Exception as e:
        return jsonify({"message": str(e)}), 500

if __name__ == "__main__":
    app.run()

In [ ]:
# Setup ngrok for public URL (Optional - for external access)
from pyngrok import ngrok
import getpass

# Get ngrok auth token (sign up at https://ngrok.com for free)
print("Get your free ngrok token from: https://dashboard.ngrok.com/get-started/your-authtoken")
ngrok_token = getpass.getpass("Enter your ngrok auth token (or press Enter to skip): ")

if ngrok_token:
    ngrok.set_auth_token(ngrok_token)
    public_url = ngrok.connect(5000)
    print(f"\n✅ Public URL: {public_url}")
    print(f"Use this URL in your frontend: {public_url}")
else:
    print("\n⚠ Skipping ngrok. Server will run locally only.")

In [ ]:
# Run Flask server
from threading import Thread
import time

def run_flask():
    from app import app
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start Flask in background thread
thread = Thread(target=run_flask)
thread.daemon = True
thread.start()

time.sleep(3)
print("\n🚀 Flask server is running!")
print("\nTest endpoints:")
print("- GET  /")
print("- GET  /disaster")
print("- GET  /weather?location=Mumbai")
print("- GET  /disaster-prediction?location=Mumbai")
print("\nServer will keep running. Don't close this cell!")

In [ ]:
# Test the API
import requests

# Test home endpoint
response = requests.get("http://localhost:5000/")
print("Home:", response.json())

# Test disaster prediction
response = requests.get("http://localhost:5000/disaster-prediction?location=Mumbai")
print("\nDisaster Prediction:", response.json())